In [1]:
# some example data
import pandas as pd
from typing import Union
from statsmodels.tsa.vector_ar.var_model import VARResultsWrapper
from statsmodels.tsa.api import VAR

from statsmodels.tsa.base.datetools import dates_from_str

In [34]:
data = pd.read_csv(dir+"National Stock Exchange/infy_stock.csv")

In [35]:
"""
要把 dataframe 的格式清理成以下格式
dates_from_str 要放入 index
剩下的資料只能保留 feature
"""
# statsmodels的範例資料
import statsmodels.api as sm
mdata = sm.datasets.macrodata.load_pandas().data
mdata
dates = mdata[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
quarterly = dates_from_str(quarterly)
mdata.index = pd.DatetimeIndex(quarterly)
data = mdata.drop(columns=["year","quarter"])
data

# KAGGLE 範例資料 1 
dir = "C:/Users/foresight_User/Desktop/公司文件/9.產品/AIUPS/"
data = pd.read_csv(dir+"National Stock Exchange/infy_stock.csv")
data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
data = data.drop(columns=["Date","Symbol","Series"])
data

# KAGGLE 範例資料 2
data = pd.read_csv(dir+"National Stock Exchange/nifty_it_index.csv")
data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
data = data.drop(columns=["Date"])
data

# KAGGLE 範例資料 3
data = pd.read_csv(dir+"National Stock Exchange/tcs_stock.csv")
data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
data = data.drop(columns=["Date","Symbol","Series"])
data

,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
2015-01-01,2558.25,2567.00,2567.00,2541.00,2550.00,2545.55,2548.51,183415,4.674345e+13,8002,52870,0.2883
2015-01-02,2545.55,2551.00,2590.95,2550.60,2588.40,2579.45,2568.19,462870,1.188740e+14,27585,309350,0.6683
2015-01-05,2579.45,2581.00,2599.90,2524.65,2538.10,2540.25,2563.94,877121,2.248886e+14,43234,456728,0.5207
2015-01-06,2540.25,2529.10,2529.10,2440.00,2450.05,2446.60,2466.90,1211892,2.989615e+14,84503,714306,0.5894
2015-01-07,2446.60,2470.00,2479.15,2407.45,2426.90,2417.70,2433.96,1318166,3.208362e+14,101741,886368,0.6724
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-24,2425.80,2421.00,2438.20,2404.00,2435.00,2434.25,2424.03,421580,1.021924e+14,21932,188670,0.4475
2015-12-28,2434.25,2428.00,2466.40,2420.20,2456.95,2462.70,2455.00,1852099,4.546896e+14,43390,1060100,0.5724
2015-12-29,2462.70,2458.35,2465.30,2445.75,2449.35,2455.80,2452.26,854262,2.094875e+14,26975,433986,0.5080
2015-12-30,2455.80,2453.05,2459.00,2412.30,2421.70,2418.30,2442.06,802881,1.960682e+14,49464,632180,0.7874


In [2]:
def time_series_format_preprocessing(df:pd.DataFrame, datetime_col:str):
    #
    # YS(年初), MS(月初), W(周), D(日), H(小時), T(分鐘), S(秒),
    #
    # df = df.copy()
    # df[datetime_col] = pd.to_datetime(df[datetime_col], format='%Y-%m-%d %H:%M:%S')
    # df = df.set_index(datetime_col)
    # df = df.asfreq(interval)
    # return df if set_index_flag else df.reset_index(drop=False)
    df.index =pd.DatetimeIndex(dates_from_str(data[datetime_col]))
    df = df.drop(columns=[datetime_col])
    """
    Imputes missing values in a time series DataFrame using the specified function.

    Args:
        df (pd.DataFrame): The input DataFrame containing the time series data.
        func (str): The imputation function to be applied. Supported options are:
                    - 'forward_fill': Forward fill missing values.
                    - 'backward_fill': Backward fill missing values.
                    - 'moving_average': Impute missing values using moving average.
                    - 'interpolation': Perform linear interpolation to fill missing values.
        col_name (str): The name of the column to impute missing values.

    Returns:
        pd.DataFrame: The DataFrame with missing values imputed based on the specified function.
    """
    return df


def time_series_impute_missing_value(df, func, col_name):
    """
    Imputes missing values in a time series DataFrame using the specified function.

    Args:
        df (pd.DataFrame): The input DataFrame containing the time series data.
        func (str): The imputation function to be applied. Supported options are:
                    - 'forward_fill': Forward fill missing values.
                    - 'backward_fill': Backward fill missing values.
                    - 'moving_average': Impute missing values using moving average.
                    - 'interpolation': Perform linear interpolation to fill missing values.
        col_name (str): The name of the column to impute missing values.

    Returns:
        pd.DataFrame: The DataFrame with missing values imputed based on the specified function.
    """
    df = df.copy()
    if func == 'forward_fill':
        df[col_name].ffill(inplace=True)
    elif func == 'backward_fill':
        df[col_name].bfill(inplace=True)
    elif func == 'moving_average':
        df[col_name].fillna(df[col_name].rolling(window=3, min_periods=1).mean(), inplace=True)
    elif func == 'interpolation':
        df[col_name].interpolate(inplace=True)
    else:
        raise ValueError(f"Unsupported imputation function: {func}")

    return df

In [3]:
def cal_maxLag(data:pd.DataFrame):
    n_totobs = len(data)
    ntrend = 1 #len(trend) if trend.startswith("c") else 0
    neqs = data.shape[1]
    max_estimable = (n_totobs - neqs - ntrend) // (1 + neqs)
    return max_estimable


maxlags = cal_maxLag(data) # USER 不能指定超過這個的LAG，LAG值也要大於0

NameError: name 'data' is not defined

In [4]:

def vectorAutoregression(data:pd.DataFrame,maxlags:Union[int,str]="auto",ic:str=None):
    model = VAR(data)
    # ==== 這邊不要動 =====

    """ 
    這是套件設定的
    trend : str {"n", "c", "ct", "ctt"}
        * "n" - no deterministic terms
        * "c" - constant term
        * "ct" - constant and linear term
        * "ctt" - constant, linear, and quadratic term

    maxlags 不可以超過 max_estimable 的值
    maxlags 為模型擬合最大數值
    statemodel有設定條件，已經寫在下述的程式
    使用者要調整低於 maxlags
    """
    max_estimable = cal_maxLag(data)
    # ==== 這邊不要動 =====

    print(" maxlags 要小於等於: ", max_estimable)

    if maxlags == "auto":
        maxlags = max_estimable
    if maxlags > max_estimable:
        raise Exception(" maxlags 要小於等於: ", max_estimable)


    """
    ic 為評估模型的好壞
    ic = {'aic', 'fpe', 'hqic', 'bic', None}
    Information criterion to use for VAR order selection.
    aic : Akaike
    fpe : Final prediction error
    hqic : Hannan-Quinn
    bic : Bayesian a.k.a. Schwarz
    """

    results = model.fit(maxlags=maxlags, ic=ic)
    print(f"在最大 lag 數目為 {max_estimable} 的情況下，VAR 找出的最佳 lag 為: ",results.k_ar)
    return results

In [5]:
def vectorAutoregressionRelationship(results:VARResultsWrapper,target:str,pvalue_threshold:float=0.05):
    # target  客人關心的 Y 是甚麼，Y 會包含在 results 中

    coef_df = results.params[target]
    pvalues_df = results.pvalues[target]

    # 合并系数和p值
    summary = pd.concat([coef_df, pvalues_df], axis=1)
    summary.columns = ['coef', 'pvalue']
    summary = summary.drop(index="const").reset_index()
    summary_index = summary["index"].str.split(".", expand=True).rename(columns={0:"time_lag",1:"feature"})
    summary_index["time_lag"] = summary_index["time_lag"].str.replace("L","").astype(int)
    summary = pd.concat([summary_index,summary],axis=1).drop(columns="index")
    summary = summary[summary["pvalue"]<pvalue_threshold].reset_index(drop=True)
    return summary

In [17]:
data = pd.read_csv("C:/Users/foresight_User/Data/h2o_exog.csv") #fecha Date

In [13]:
#data = pd.read_csv(dir+"National Stock Exchange/tcs_stock.csv").drop(columns=["Symbol","Series"])

#data = time_series_impute_missing_value(data, 'interpolation', "Volume")

In [29]:
data = pd.read_csv("C:/Users/foresight_User/Data/h2o_exog.csv") #fecha Date
data = time_series_format_preprocessing(data,"fecha")

# data = pd.read_csv("C:/Users/foresight_User/Data/Chiller_CH14(01~24).csv").drop(columns=["CONTEXTID"]) #TIMETAG
# data = time_series_format_preprocessing(data,"TIMETAG")

In [30]:
# 檢查是否有負值並新增相應欄位，取絕對值
for col in data.columns:
    data[f"{col}_isNegative"] = (data[col] < 0).astype(int)
    if sum(data[f"{col}_isNegative"]) ==0:
        data = data.drop(columns=[f"{col}_isNegative"])
    else:
        data[col] = data[col].abs()

In [31]:
data

,y,exog_1,exog_2
1992-04-01,0.379808,0.958792,1.166029
1992-05-01,0.361801,0.951993,1.117859
1992-06-01,0.410534,0.952955,1.067942
1992-07-01,0.483389,0.958078,1.097376
1992-08-01,0.475463,0.956370,1.122199
...,...,...,...
2008-02-01,0.761822,1.515840,1.786373
2008-03-01,0.649435,1.506258,1.694264
2008-04-01,0.827887,1.505253,1.627135
2008-05-01,0.816255,1.491464,1.555068


In [38]:
"""
讓使用者決定好奇的target是甚麼，套件會自動找出所有跟target相關的不同time lag的時間變相
maxlags 會影響最終最佳推薦的結果，也得慎選，但設定上不可以超過 cal_maxLag 輸出的值
"""

#data = time_series_format_preprocessing(data,"Date")
#data = time_series_impute_missing_value(data, 'interpolation', "Volume")
maxlags = cal_maxLag(data) # USER 不能指定超過這個的LAG，LAG值也要大於0
target = "y"
results = vectorAutoregression(data,maxlags=9,ic=None) 
pvalue_threshold = 0.05
VAR_relationship = vectorAutoregressionRelationship(results=results,target=target,pvalue_threshold=0.05)

 maxlags 要小於等於:  47
在最大 lag 數目為 47 的情況下，VAR 找出的最佳 lag 為:  9


c:\Users\foresight_User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\foresight_User\anaconda3\lib\site-packages\statsmodels\tsa\vector_ar\var_model.py:1558: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(self.cov_params()))


In [39]:
VAR_relationship

,time_lag,feature,coef,pvalue
0,6,exog_1,-3.708813,0.003547
1,7,exog_1,2.926414,0.026044


In [30]:
vectorAutoregression(data,maxlags=3,ic=None)# 'aic', 'fpe', 'hqic', 'bic', None

 maxlags 要小於等於:  443
在最大 lag 數目為 443 的情況下，VAR 找出的最佳 lag 為:  3


c:\Users\foresight_User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\foresight_User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [49]:
"""
生成VAR關係公式
"""
"""
可有可無，看需求
"""
formula_parts = []
for index, row in VAR_relationship.iterrows():
    coef = row['coef']
    if coef < 0:
        term = f"- {-coef:.3f} * {row['feature']}(t-{row['time_lag']})"
    else:
        term = f"{coef:.3f} * {row['feature']}(t-{row['time_lag']})"
    formula_parts.append(term)

formula = " + ".join(formula_parts).replace("+ -", "- ")
target_formula = f"{target}(t) = {formula}"

# 打印公式
print(target_formula)

Trades(t) = 0.553 * Trades(t-1)
